In [1]:
from tensorflow.python.keras.applications.mobilenet import MobileNet
from tensorflow.python.keras.preprocessing import image
from tensorflow.python.keras.applications.mobilenet import preprocess_input, decode_predictions
import numpy as np
import tensorflow as tf
import urllib.request as urllib
import functools
import tensorflow.contrib.eager as tfe
tf.enable_eager_execution()

/opt/conda/envs/riptide-py3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
sess = tf.Session()

from tensorflow.python.keras import backend as K
K.set_session(sess)
#sess.run(tf.global_variables_initializer())

In [3]:
with tf.device('/cpu:0'):
    model = MobileNet(weights='imagenet')

#parallel_model = tf.keras.utils.multi_gpu_model(model, gpus=3)    
    
img_path = '/data/images/gix.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

In [19]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         ?                         0         
_________________________________________________________________
conv1 (Conv2D)               ?                         864       
_________________________________________________________________
conv1_bn (BatchNormalization ?                         128       
_________________________________________________________________
conv1_relu (Activation)      ?                         0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  ?                         288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza ?                         128       
_________________________________________________________________
conv_dw_1_relu (Activation)  ?                         0         
__________

In [4]:
preds = model.predict(x)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
print('Predicted:', decode_predictions(preds, top=3)[0])

Predicted: [('n02793495', 'barn', 0.2323404), ('n03028079', 'church', 0.21752653), ('n02859443', 'boathouse', 0.12427856)]


In [5]:
from preprocessing import inception_preprocessing
url = 'https://upload.wikimedia.org/wikipedia/commons/7/70/EnglishCockerSpaniel_simon.jpg'
image_string = urllib.urlopen(url).read()
data = tf.image.decode_jpeg(image_string, channels=3)
preprocess = functools.partial(inception_preprocessing.preprocess_image, height=224, width=224, is_training=False)
processed_image = preprocess(data)
processed_images  = tf.expand_dims(processed_image, 0)

In [6]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

#SVG(model_to_dot(model).create(prog='dot', format='svg'))

Using TensorFlow backend.


In [11]:
output = model(processed_images)
if tf.executing_eagerly():
    tf_out = output.numpy()
else:
    tf_out = sess.run([output])[0]

In [12]:
print('Predicted:', decode_predictions((tf_out), top=3)[0])

Predicted: [('n02102318', 'cocker_spaniel', 0.8370111), ('n02102480', 'Sussex_spaniel', 0.049584765), ('n02100877', 'Irish_setter', 0.032806892)]


In [17]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         ?                         0         
_________________________________________________________________
conv1 (Conv2D)               ?                         864       
_________________________________________________________________
conv1_bn (BatchNormalization ?                         128       
_________________________________________________________________
conv1_relu (Activation)      ?                         0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  ?                         288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza ?                         128       
_________________________________________________________________
conv_dw_1_relu (Activation)  ?                         0         
__________

In [10]:
from riptide.data import *

In [11]:
def model_fn(features, labels, mode, params):
    """The model_fn argument for creating an Estimator."""
    #with tf.device('/cpu:0'):
    #    orig_model = VGG16(weights=None, input_tensor=features['input_1'])
    model = MobileNet(weights=None, input_tensor=features['input_1'])
    image = features
    if isinstance(image, dict):
        image = features['input_1']
    if mode == tf.estimator.ModeKeys.PREDICT:
        logits = model(image)
        predictions = {
            'classes': tf.argmax(logits, axis=1),
            'probabilities': tf.nn.softmax(logits),
        }
        return tf.estimator.EstimatorSpec(
            mode=tf.estimator.ModeKeys.PREDICT,
            predictions=predictions,
            export_outputs={
                'classify': tf.estimator.export.PredictOutput(predictions)
            })
    if mode == tf.estimator.ModeKeys.TRAIN:
        #optimizer = tf.train.AdamOptimizer(learning_rate=1e-4)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)

        # If we are running multi-GPU, we need to wrap the optimizer.
        #if params.get('multi_gpu'):
        optimizer = tf.contrib.estimator.TowerOptimizer(optimizer)

        logits = model(image)
        loss = tf.losses.softmax_cross_entropy(onehot_labels=labels, logits=logits)
        accuracy = tf.metrics.accuracy(labels=tf.argmax(labels, axis=1), predictions=tf.argmax(logits, axis=1))
            #labels=labels, predictions=tf.argmax(logits, axis=1))
        # Name the accuracy tensor 'train_accuracy' to demonstrate the
        # LoggingTensorHook.
        tf.identity(accuracy[1], name='train_accuracy')
        tf.summary.scalar('train_accuracy', accuracy[1])
        return tf.estimator.EstimatorSpec(
            mode=tf.estimator.ModeKeys.TRAIN,
            loss=loss,
            train_op=optimizer.minimize(loss, tf.train.get_or_create_global_step()))
    if mode == tf.estimator.ModeKeys.EVAL:
        logits = model(image)
        loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
        return tf.estimator.EstimatorSpec(
            mode=tf.estimator.ModeKeys.EVAL,
            loss=loss,
            eval_metric_ops={
                'accuracy':
                    tf.metrics.accuracy(
                        labels=tf.argmax(labels, axis=1),
                        predictions=tf.argmax(logits, axis=1)),
            })


In [12]:
input_fn = functools.partial(imgs_input_fn, ImagenetData("/data2/imagenet/tfrecords", "val"), preprocess=preprocess, perform_shuffle=True, batch_size=192)

In [13]:
# check out tf.contrib.estimator.replicate_model_fn as a better? way to do parallelization
#model_estimator = tf.estimator.Estimator(model_fn = model_fn)
model_estimator = tf.estimator.Estimator(model_fn = tf.contrib.estimator.replicate_model_fn(model_fn))

INFO:tensorflow:Replicating the `model_fn` across ['/device:GPU:0', '/device:GPU:1', '/device:GPU:2'].  Variables are going to be placed on ['/CPU:0'].  Consolidation device is going to be /CPU:0.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp_prfecmj', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ef86dbcfdd8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [14]:
model_estimator.train(input_fn = input_fn)

INFO:tensorflow:Calling model_fn.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmp_prfecmj/model.ckpt.
INFO:tensorflow:loss = 6.9077554, step = 1
INFO:tensorflow:global_step/sec: 0.73533
INFO:tensorflow:loss = 6.9077554, step = 101 (135.995 sec)


KeyboardInterrupt: 